In [1]:
from __future__ import print_function
from tensorflow.keras.models import load_model
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [2]:
import codecs
batch_size = 64  
epochs = 100 
latent_dim = 256  
num_samples = 10000  
data_path = 'fra.txt'

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [4]:
with codecs.open(data_path, 'rb', encoding='utf-8',errors='ignore') as f:
    lines = f.read().split('\n')

In [5]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text= line.split('\u202f')[0].split('\t')[0]
    target_text= line.split('\u202f')[0].split('\t')[1]
    
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)



In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [10]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [11]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [12]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [13]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [14]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10000, 16, 71)
(10000, 59, 92)
(10000, 59, 92)


In [15]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [16]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 30s 4ms/step - loss: 1.1888 - accuracy: 0.7259 - val_loss: 1.1042 - val_accuracy: 0.7012
Epoch 2/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.8642 - accuracy: 0.7665 - val_loss: 0.8570 - val_accuracy: 0.7619
Epoch 3/100
8000/8000 [==============================] - 29s 4ms/step - loss: 0.6819 - accuracy: 0.8073 - val_loss: 0.7352 - val_accuracy: 0.7886
Epoch 4/100
8000/8000 [==============================] - 28s 3ms/step - loss: 0.6141 - accuracy: 0.8228 - val_loss: 0.6601 - val_accuracy: 0.8068
Epoch 5/100
8000/8000 [==============================] - 27s 3ms/step - loss: 0.5543 - accuracy: 0.8385 - val_loss: 0.6169 - val_accuracy: 0.8176
Epoch 6/100
8000/8000 [==============================] - 27s 3ms/step - loss: 0.5154 - accuracy: 0.8492 - val_loss: 0.5822 - val_accuracy: 0.8274
Epoch 7/100
8000/8000 [==============================] - 29s 4ms/step - loss

In [17]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [18]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [19]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [21]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('actual sentence:',target_texts[seq_index])


-
Input sentence: Go.
Decoded sentence: Va !

actual sentence: 	Va !

-
Input sentence: Hi.
Decoded sentence: Salut.

actual sentence: 	Salut !

-
Input sentence: Hi.
Decoded sentence: Salut.

actual sentence: 	Salut.

-
Input sentence: Run!
Decoded sentence: Cours

actual sentence: 	Cours

-
Input sentence: Run!
Decoded sentence: Cours

actual sentence: 	Courez

-
Input sentence: Who?
Decoded sentence: Qui ?

actual sentence: 	Qui ?

-
Input sentence: Wow!
Decoded sentence: Ça alors

actual sentence: 	Ça alors

-
Input sentence: Fire!
Decoded sentence: Au feu !

actual sentence: 	Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide

actual sentence: 	À l'aide

-
Input sentence: Jump.
Decoded sentence: Saute.

actual sentence: 	Saute.

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

actual sentence: 	Ça suffit

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

actual sentence: 	Stop

-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

actual sentence: 	Arrête-

-
Input sentence: Call me.
Decoded sentence: Appellez-moi !

actual sentence: 	Appellez-moi !

-
Input sentence: Call us.
Decoded sentence: Appelez-nous !

actual sentence: 	Appelle-nous !

-
Input sentence: Call us.
Decoded sentence: Appelez-nous !

actual sentence: 	Appelez-nous !

-
Input sentence: Come in.
Decoded sentence: Entrez

actual sentence: 	Entrez



In [23]:
model.save('model_weights.h5')
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')